# VIRTUAL MEDIC CHATBOT FOR DISEASE AND DRUG CLASSIFICATION

## PRESENTED BY:

## Shyam R -19MIC0017
## Niketha S-19MIC0035
## Nivethitha S-19MIC0030
## Ram Gnaneshwaran -19MIC0104
## Nihaal Ahmed -19MIC0038

## AIM:

## In this project, we aim to build an end-to-end open source information retriveval medical chatbot
## Our chatbot incorporates best practices in IR with a transformer-based reader to give answers quickly and efficiently from a huge corpus of medically related data.


In [1]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split,cross_val_score
import math
import operator
import pickle
import re
from nltk.stem import WordNetLemmatizer
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from statistics import mean
from nltk.corpus import wordnet 
import requests
from bs4 import BeautifulSoup
from itertools import combinations
from time import time
from collections import Counter
import operator
import warnings
from Treatment import diseaseDetail
warnings.simplefilter("ignore")

ImportError: cannot import name 'search' from 'googlesearch' (C:\Users\nikis\AppData\Local\Programs\Python\Python310\lib\site-packages\googlesearch\__init__.py)

In [2]:
pip upgrsde

Note: you may need to restart the kernel to use updated packages.Collecting google-search
  Using cached google_search-1.1.1-py2.py3-none-any.whl (6.5 kB)
     ---------------------                    1.9/3.6 MB 6.5 kB/s eta 0:04:16



ERROR: Wheel 'lxml' located at C:\Users\nikis\AppData\Local\Temp\pip-unpack-94lbm_ff\lxml-4.9.1-cp310-cp310-win_amd64.whl is invalid.


In [4]:
pip install google-search

  Using cached google_search-1.1.1-py2.py3-none-any.whl (6.5 kB)
     -------------------                      1.8/3.6 MB 59.1 kB/s eta 0:00:31
Note: you may need to restart the kernel to use updated packages.


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\nikis\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 437, in _error_catcher
    yield
  File "C:\Users\nikis\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 560, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\nikis\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 526, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\Users\nikis\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "C:\Users\nikis\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 465, in read
    s = self.fp.read(amt)
  File "C:\Users\nikis\AppData\Local\Programs\Python\Python310\lib\socket.py", line 705, in readint

In [1]:
pip install google-search

Note: you may need to restart the kernel to use updated packages.


In [4]:
#from toolkit.interface import interface

In [9]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [5]:
df=pd.read_csv(r"C:\Users\nikis\OneDrive\Desktop\Virtual Medical Chatbot_NLP J Comp\Dataset\dis_sym_dataset_norm.csv")
documentname_list=list(df['label_dis'])
df=df.iloc[:,1:]
columns_name=list(df.columns)
documentname_list=list(documentname_list)

N=len(df)
M=len(columns_name)
idf={}
for col in columns_name:
  temp=np.count_nonzero(df[col])
  idf[col]=np.log(N/temp)
tf={}
for i in range(N):
  for col in columns_name:
    key=(documentname_list[i],col)
    tf[key]=df.loc[i,col]
tf_idf={}
for i in range(N):
  for col in columns_name:
    key=(documentname_list[i],col)
    tf_idf[key]=float(idf[col])*float(tf[key])

D = np.zeros((N, M),dtype='float32')
for i in tf_idf:
    sym = columns_name.index(i[1])
    dis=documentname_list.index(i[0])
    D[dis][sym] = tf_idf[i]

def cosine_dot(a, b):
    if np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
        return 0
    else:
        temp = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
        return temp


def convert_tolowercase(data):
    return data.lower()


def regextokenizer_func(data):
    tokenizer = RegexpTokenizer(r'\w+')
    data = tokenizer.tokenize(data)
    return data


def gen_vector(tokens):
    Q = np.zeros(M)
    counter = Counter(tokens)
    query_weights = {}
    for token in np.unique(tokens):
        tf = counter[token]
        try:
          idf_temp=idf[token]
        except:
          pass
        try:
            ind = columns_name.index(token)
            Q[ind] = tf*idf_temp
        except:
            pass
    return Q


def tf_idf_score(k, query):
    query_weights = {}
    for key in tf_idf:
        if key[1] in query:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
  
    l = []
    for i in query_weights[:k]:
        l.append(i)
    return l

 
def cosine_similarity(k, query):
    d_cosines = []
    query_vector = gen_vector(query)
    for d in D:
        d_cosines.append(cosine_dot(query_vector, d))
    out = np.array(d_cosines).argsort()[-k:][::-1]
  
    final_display_disease={}
    for lt in set(out):
      final_display_disease[lt] = float(d_cosines[lt])
    return final_display_disease

In [6]:

def synonyms(term):
    synonyms = []
    response = requests.get('https://www.thesaurus.com/browse/{}'.format(term))
    soup = BeautifulSoup(response.content,  "html.parser")
    try:
        container=soup.find('section', {'class': 'MainContentContainer'}) 
        row=container.find('div',{'class':'css-191l5o0-ClassicContentCard'})
        row = row.find_all('li')
        for x in row:
            synonyms.append(x.get_text())
    except:
        None
    for syn in wordnet.synsets(term):
        synonyms+=syn.lemma_names()
    return set(synonyms)

In [7]:
splitter = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [8]:
df_comb = pd.read_csv(r"C:\Users\nikis\OneDrive\Desktop\Disease-Detection-based-on-Symptoms-master\Dataset\dis_sym_dataset_comb.csv") # Disease combination
df_norm = pd.read_csv(r"C:\Users\nikis\OneDrive\Desktop\Disease-Detection-based-on-Symptoms-master\Dataset\dis_sym_dataset_norm.csv") # Individual Disease
Y = df_norm.iloc[:, 0:1]
X = df_norm.iloc[:, 1:]
dataset_symptoms = list(X.columns)
diseases = list(set(Y['label_dis']))
diseases.sort()

In [9]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nikis\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
print("Hello! Welcome to our Virtual-Chatbot")
user_symptoms = str(input("\nEnter symptoms:\n")).lower().split(',')
processed_user_symptoms=[]
for sym in user_symptoms:
    sym=sym.strip()
    sym=sym.replace('-',' ')
    sym=sym.replace("'",'')
    sym = ' '.join([lemmatizer.lemmatize(word) for word in splitter.tokenize(sym)])
    processed_user_symptoms.append(sym)  

In [11]:
user_symptoms = []
for user_sym in processed_user_symptoms:
    user_sym = user_sym.split()
    str_sym = set()
    for comb in range(1, len(user_sym)+1):
        for subset in combinations(user_sym, comb):
            subset=' '.join(subset)
            subset = synonyms(subset) 
            str_sym.update(subset)
    str_sym.add(' '.join(user_sym))
    user_symptoms.append(' '.join(str_sym).replace('_',' '))
print("Analysing the Symptoms entered")
print(user_symptoms)

After query expansion done by using the symptoms entered
['pyrexia fever feverishness febricity febrility']


In [12]:
found_symptoms = set()
for idx, data_sym in enumerate(dataset_symptoms):
    data_sym_split=data_sym.split()
    for user_sym in user_symptoms:
        count=0
        for symp in data_sym_split:
            if symp in user_sym.split():
                count+=1
        if count/len(data_sym_split)>0.5:
            found_symptoms.add(data_sym)
found_symptoms = list(found_symptoms)

In [13]:
print("Matching symptoms from the data entered")
for idx, symp in enumerate(found_symptoms):
    print(idx,":",symp)

select_list = input("\n Hello User! Select more relevant symptoms \n").split()

dis_list = set()
final_symp = [] 
counter_list = []
for idx in select_list:
    symp=found_symptoms[int(idx)]
    final_symp.append(symp)
    dis_list.update(set(df_norm[df_norm[symp]==1]['label_dis']))
   
for dis in dis_list:
    row = df_norm.loc[df_norm['label_dis'] == dis].values.tolist()
    row[0].pop(0)
    for idx,val in enumerate(row[0]):
        if val!=0 and dataset_symptoms[idx] not in final_symp:
            counter_list.append(dataset_symptoms[idx])
             
dict_symp = dict(Counter(counter_list))
dict_symp_tup = sorted(dict_symp.items(), key=operator.itemgetter(1),reverse=True)   

Top matching symptoms from your search!
0 : fever

Please select the relevant symptoms. Enter indices (separated-space):
0


In [14]:
found_symptoms=[]
count=0
for tup in dict_symp_tup:
    count+=1
    found_symptoms.append(tup[0])
    if count%5==0 or count==len(dict_symp_tup):
        print("\nCommon symptoms:")
        for idx,ele in enumerate(found_symptoms):
            print(idx,":",ele)
        select_list = input("Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:\n").lower().split();
        if select_list[0]=='no':
            break
        if select_list[0]=='-1':
            found_symptoms = [] 
            continue
        for idx in select_list:
            final_symp.append(found_symptoms[int(idx)])
        found_symptoms = []    


Common co-occuring symptoms:
0 : headache
1 : testicular pain
2 : vomiting
3 : barky cough
4 : sore throat
Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:
3

Common co-occuring symptoms:
0 : maculopapular rash
1 : diarrhea
2 : confusion
3 : swollen lymph node
4 : feeling tired
Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:
-1

Common co-occuring symptoms:
0 : runny nose
1 : unintended weight loss
2 : shortness breath
3 : muscle weakness
4 : chest pain
Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:
n0


ValueError: invalid literal for int() with base 10: 'n0'

In [15]:
k = 10

print("Final list of Symptoms used for prediction are : ")
for val in final_symp:
    print(val)

Final list of Symptoms used for prediction are : 
fever
barky cough


In [16]:
topk1=tf_idf_score(k,final_symp)
topk2=cosine_similarity(k,final_symp)
print(f"\nTop {k} diseases predicted based on TF_IDF Matching :\n")
i = 0
topk1_index_mapping = {}
for key, score in topk1:
  print(f"{i}. Disease : {key} \t Score : {round(score, 2)}")
  topk1_index_mapping[i] = key
  i += 1

select = input("\nTo enter more details about the disease? Enter index of disease or '-1' to discontinue:\n")
if select!='-1':
    dis=topk1_index_mapping[int(select)]
    print()
    print(diseaseDetail(dis))


Top 10 diseases predicted based on TF_IDF Matching :

0. Disease : Common cold 	 Score : 4.55
1. Disease : Coronavirus disease 2019 (COVID-19) 	 Score : 4.55
2. Disease : Cough 	 Score : 4.55
3. Disease : Diphtheria 	 Score : 4.55
4. Disease : Legionellosis 	 Score : 4.55
5. Disease : Measles 	 Score : 4.55
6. Disease : Middle East respiratory syndrome coronavirus (MERS‐CoV) 	 Score : 4.55
7. Disease : Nipah virus infection 	 Score : 4.55
8. Disease : Pneumonia 	 Score : 4.55
9. Disease : Asbestos-related diseases 	 Score : 3.26

More details about the disease? Enter index of disease or '-1' to discontinue:
0



HTTPError: HTTP Error 429: Too Many Requests

In [ ]:
print(f"Top {k} disease based on Cosine Similarity Matching :\n ")
topk2_sorted = dict(sorted(topk2.items(), key=lambda kv: kv[1], reverse=True))
j = 0
topk2_index_mapping = {}
for key in topk2_sorted:
  print(f"{j}. Disease : {diseases[key]} \t Score : {round(topk2_sorted[key], 2)}")
  topk2_index_mapping[j] = diseases[key]
  j += 1

    
select = input("\nMore details about the disease? Enter index of disease or '-1' to discontinue and close the system:\n")
if select!='-1':
    dis=topk2_index_mapping[int(select)]
    print()
    print(diseaseDetail(dis))

Top 10 disease based on Cosine Similarity Matching :
 
0. Disease : Middle East respiratory syndrome coronavirus (MERS‐CoV) 	 Score : 0.79
1. Disease : Diphtheria 	 Score : 0.73
2. Disease : Nipah virus infection 	 Score : 0.68
3. Disease : Legionellosis 	 Score : 0.67
4. Disease : Cough 	 Score : 0.66
5. Disease : Common cold 	 Score : 0.56
6. Disease : Pneumonia 	 Score : 0.43
7. Disease : Asbestos-related diseases 	 Score : 0.42
8. Disease : Measles 	 Score : 0.42
9. Disease : Coronavirus disease 2019 (COVID-19) 	 Score : 0.36
